In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import pickle 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold


from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score



In [2]:
C=1.0
n_splits=5
dv=DictVectorizer(sparse=False)

In [3]:
df=pd.read_csv("/home/leviscoffie/MLzoomcamp/Telco-Churn-Prediction-MLzoomcamp-/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.columns.str.replace(' ','_').str.lower()
df.columns=df.columns.str.replace(' ','_').str.lower()

categorical_columns=list(df.dtypes[df.dtypes=='object'].index)

categorical_columns

for c in categorical_columns:
    df[c]= df[c].str.lower().str.replace(' ','_')

df.totalcharges=pd.to_numeric(df.totalcharges, errors="coerce")
df.totalcharges=df.totalcharges.fillna(0)


df.churn=(df.churn=='yes').astype(int)


In [4]:
df_full_train, df_test= train_test_split(df, test_size=0.2, random_state=1)


In [5]:
df_train, df_val= train_test_split(df_full_train, test_size=0.25, random_state=1)

In [6]:
numerical=['tenure','monthlycharges','totalcharges']

categorical= ['gender', 'seniorcitizen', 'partner', 'dependents','phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [7]:
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)


In [8]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [9]:
y_train=df_train.churn.values
y_val=df_val.churn.values
y_test=df_test.churn.values

y_full_train= df_full_train.churn.values

In [10]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [11]:
def train(df_train,y_train, C=1.0):
    dicts=df_train[categorical+numerical].to_dict(orient='records')
                                   
    dv=DictVectorizer(sparse=False)
    
    
    X_train=dv.fit_transform(dicts)
    
    model=LogisticRegression(solver='liblinear', max_iter=10000 )
    model.fit(X_train,y_train)
    
    return dv, model

In [12]:
dv, model=train(df_train,y_train, C=1.0)

In [13]:
# call another function that makes predictions
def predict(df,dv,model):
    dicts=df[categorical+numerical].to_dict(orient='records')
    X=dv.transform(dicts)
    y_pred=model.predict_proba(X)[:,1]
    
    return y_pred

In [14]:
y_pred=predict(df_val,dv, model)

In [15]:
# Now calling K-fold cross validation from sklearn

kf=KFold(n_splits=n_splits, shuffle=True, random_state=1) 

In [16]:
kf.split(df_full_train)

train_idx, val_idx=next(kf.split(df_full_train))

len(train_idx), len(val_idx)

(4507, 1127)

In [17]:
for train_idx, val_idx in kf.split(df_full_train):

    scores=[]
    
    df_train=df_full_train.iloc[train_idx]
    df_val=df_full_train.iloc[val_idx]

    y_train=df_train.churn.values
    y_val=df_val.churn.values

    dv, model=train(df_train,y_train, C=1.0)
    y_pred=predict(df_val,dv, model)

    auc=roc_auc_score(y_val, y_pred)

    scores.append(auc)

print("C=%s %.3f +- %.3f"%(C, np.mean(scores),np.std(scores))) 

C=1.0 0.852 +- 0.000


In [18]:
dv, model=train(df_full_train,df_full_train.churn.values, C=1.0)
y_pred=predict(df_test,dv, model)

auc=roc_auc_score(y_test, y_pred)

auc

0.8579400803839363

### Saving the Model

In [19]:
import pickle 

In [20]:
output_file=f"model_C={C}.bin"

output_file

'model_C=1.0.bin'

In [21]:
f_out=open(output_file,'wb')
pickle.dump((dv,model), f_out)

f_out.close()

In [22]:
#Alternativey for the code cell above, we can do this:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [ ]:
import os
os._exit(00)

## Load the model 

In [1]:
model_file='model_C=1.0.bin'

In [2]:
import pickle

with open(model_file, 'rb') as f_in:
    dv, model=pickle.load(f_in)

In [3]:
dv, model

(DictVectorizer(sparse=False),
 LogisticRegression(max_iter=10000, solver='liblinear'))

In [4]:
#Picking one customer from the dataset
# to evaluate
kastama={'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'no',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'n0_phone_service', 
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'no',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'mailed_check',
 'tenure': 1,
 'monthlycharges': 69.25,
 'totalcharges': 69.25}

In [5]:
X=dv.transform([kastama])

In [6]:
model.predict_proba(X)

array([[0.68019886, 0.31980114]])